# 08/02 train-rmse:0.233459;	valid-rmse:0.242779
# 07/31 train-rmse:0.242975;	valid-rmse:0.249397

- **V** Number of characters in title
- **V** Number of characters in description
- **V** tfidf of description
- **V** tfidf of title
- Param_1,2,3 if has param?



- Number of words
- Number of numbers. i.e. 2010
- Average Word Length
- Number of stopwords
- Number of special characters

- if the description is empty (replace with 'NA' string is fine. and then create another boolean feature)
- try new features to see if the result improved
- search for keywords in description of title. i.e. "new" or "old"
- empty description will be 0 after tfidf?
- check the predicted value, see if they fall into [0,1]

In [1]:
import pandas as pd
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import numpy as np

In [2]:
# load dataset
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
train["item_id"].is_unique

True

## Get relevant columns

In [4]:
def select_columns(data, *columns):
    return data.loc[:, columns]

#train=train[["item_id","title","description","deal_probability"]]
#test=test[["item_id","title","description"]]

## Checking missing values

In [5]:
s=train.isnull()
missing_counts_train = s[s == True].count()
s=test.isnull()
missing_counts_test = s[s == True].count()
missing_counts_train

item_id                      0
user_id                      0
region                       0
city                         0
parent_category_name         0
category_name                0
param_1                  61576
param_2                 654542
param_3                 862565
title                        0
description             116276
price                    85362
item_seq_number              0
activation_date              0
user_type                    0
image                   112588
image_top_1             112588
deal_probability             0
dtype: int64

In [6]:
missing_counts_test

item_id                      0
user_id                      0
region                       0
city                         0
parent_category_name         0
category_name                0
param_1                  22910
param_2                 233229
param_3                 306331
title                        0
description                  0
price                    30585
item_seq_number              0
activation_date              0
user_type                    0
image                    42609
image_top_1              42609
dtype: int64

In the test dataset, there is not missing values in *description*. In other words, if *description* is empty or not might not be a good features?

In [7]:
def fix_missing_desc(data):
    data['description']=data['description'].fillna('NA')
    return data

def fix_missing_title(data):
    data['title']=data['title'].fillna('NA')
    return data

## Add features

In [8]:
def add_has_description(data):
    def check_na(s):
        if s=="NA":
            return 0
        else:
            return 1
        
    data['has_desc'] = data["description"].apply(check_na)
    return data

In [9]:
def add_char_len_title(data):
    data['char_len_title'] = data["title"].apply(lambda x: len(str(x)))
    return data

def add_char_len_desc(data):
    data['char_len_desc'] = data["description"].apply(lambda x: len(str(x)))
    return data

In [10]:
train.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.00000,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.00000,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.00000,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.00000,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.00000,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.00000,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.00000,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.00000,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.00000,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.00000,0.20797


In [11]:
import nltk
from nltk.corpus import stopwords
stopWords = stopwords.words('russian')

In [12]:
# need to change data type from object to unicode
train["description"].dtype

dtype('O')

## Lowercase

In [13]:
def change_lowercase_title(data):
    data["title"] = data["title"].str.lower()
    return data

def change_lowercase_desc(data):
    data["description"] = data["description"].str.lower()
    return data

## Remove stopwords

In [14]:
#def remove_stopwords_title(data, stopwords):
#    data["title"] = data["title"].str.lower()
#    return data

#def change_des_lowercase(data):
#    data["description"] = data["description"].str.lower()
#    return data

## Tokenized

## Morphological analyzer for Russian language (3hr)

In [15]:
def add_normalize_morph_title(data):
    import re
    import pymorphy2
    morph = pymorphy2.MorphAnalyzer()
    retoken = re.compile(r'[\'\w\-]+')
    
    def get_words(s):
        list_words = retoken.findall(s)
        return ' '.join([morph.parse(x)[0].normal_form for x in list_words])  
    
    data["title_norm"] = data["title"].apply(get_words)
    return data

def add_normalize_morph_desc(data):
    import re
    import pymorphy2
    morph = pymorphy2.MorphAnalyzer()
    retoken = re.compile(r'[\'\w\-]+')
    
    def get_words(s):
        list_words = retoken.findall(s)
        return ' '.join([morph.parse(x)[0].normal_form for x in list_words])  
    
    data["desc_norm"] = data["description"].apply(get_words)
    return data

#train['title'] = train['title'].astype(str)
#train['description'] = train['description'].astype(str)

#train['title'] = train['title'].apply(normalize)
#train['description'] = train['description'].apply(normalize)

In [16]:
add_normalize_morph_desc(train[:5])

ModuleNotFoundError: No module named 'pymorphy2'

## TF-IDF

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# tfidf for title
def add_tfidf_title(data, num=100):
    tfidf = TfidfVectorizer(max_features=100, stop_words = stopWords)
    tfidf_title_train = np.array(tfidf.fit_transform(data['title'].values.astype('U')).todense(), dtype=np.float16)
    for i in range(num):
        data['tfidf_title_' + str(i)] = tfidf_title_train[:, i]
    return data

# tfidf for description
def add_tfidf_desc(data, num=100):
    tfidf = TfidfVectorizer(max_features=100, stop_words = stopWords)
    tfidf_des_train = np.array(tfidf.fit_transform(data['description'].values.astype('U')).todense(), dtype=np.float16)
    for i in range(num):
        data['tfidf_des_' + str(i)] = tfidf_des_train[:, i]
    return data

In [18]:
train.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.00000,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.00000,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.00000,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.00000,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.00000,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.00000,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.00000,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.00000,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.00000,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.00000,0.20797


## Reusable Pipeline

In [19]:
def process_data(data):

    data = (
        data
        # Clean Data
        .pipe(fix_missing_title)
        .pipe(fix_missing_desc)
        .pipe(change_lowercase_title)
        .pipe(change_lowercase_desc)
        
        # Transform data
        .pipe(select_columns, 
              "item_id",
              "title",
              "description",
              "deal_probability"
             )
        .pipe(add_char_len_title)
        .pipe(add_char_len_desc)
        .pipe(add_tfidf_title)
        .pipe(add_tfidf_desc)
        #.pipe(add_keywords_desc)
        .pipe(add_has_description)
    ) 
    
    # Return predictors and response variables separately
    X = data.drop(['deal_probability'], axis = 1)
    y = data['deal_probability'] # all nan if not exist 
    
    return X, y

In [20]:
X_train, y_train = process_data(train)

In [21]:
X_test, y_test = process_data(test)

/Users/zexuanxu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:870: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_lowerdim(tup)


## split train and validation dataset

In [22]:
X_train = X_train.drop(['item_id','title','description'],axis=1)
X_test = X_test.drop(['item_id','title','description'],axis=1)

In [23]:
X_train.to_csv('NLP_X_train.csv')
X_test.to_csv('NLP_X_test.csv')
y_train.to_csv('NLP_y_train.csv')
y_test.to_csv('NLP_y_test.csv')


In [23]:
# split train and test dataset
from sklearn.model_selection import train_test_split

# for training and validating model
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=99)


print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1202739, 212) (300685, 212) (1202739,) (300685,)


In [26]:
X_train = X_train.drop(['has_desc'],axis=1)
X_val = X_val.drop(['has_desc'],axis=1)

In [ ]:
import xgboost as xgb

# grid search result
params = {'eta': 0.3,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 1400,  
          'max_depth': 0, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'objective': 'reg:logistic', 
          'eval_metric': 'rmse', 
          'random_state': 99, 
          'silent': True}

tr_data = xgb.DMatrix(X_train, y_train)
va_data = xgb.DMatrix(X_val, y_val)
#del X_train
#del X_val
#del y_train
#del y_val
import gc
gc.collect()

watchlist = [(tr_data, 'train'), (va_data, 'valid')]

model = xgb.train(params, tr_data, 1000, watchlist, maximize=False, early_stopping_rounds = 25, verbose_eval=5)

[0]	train-rmse:0.359706	valid-rmse:0.360279
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 25 rounds.
[5]	train-rmse:0.24826	valid-rmse:0.251583
[10]	train-rmse:0.24023	valid-rmse:0.245175
[15]	train-rmse:0.238014	valid-rmse:0.24448
[20]	train-rmse:0.236339	valid-rmse:0.24443
[25]	train-rmse:0.23504	valid-rmse:0.244482
[30]	train-rmse:0.23375	valid-rmse:0.244594
[35]	train-rmse:0.232547	valid-rmse:0.244709
[40]	train-rmse:0.231484	valid-rmse:0.244835


In [ ]:
X_test.head()

In [ ]:
# generate result for test dataset

X_test_dmatrix = xgb.DMatrix(X_test)
y_pred = model.predict(X_test_dmatrix, ntree_limit=model.best_ntree_limit)
sub = pd.read_csv('data/sample_submission2.csv')
sub['deal_probability'] = y_pred
sub['deal_probability'].clip(0.0, 1.0, inplace=True)
sub.to_csv('data/nlp.csv', index=False)
sub.head()

In [ ]:
# plotting 

#from xgboost import plot_importance
#import matplotlib.pyplot as plt
#plot_importance(model)
#plt.gcf().savefig('feature_importance_xgb.png')


## Translate top words
А а, Б б, В в, Г г, Д д, Е е, Ё ё, Ж ж, З з, И и, Й й, К к, Л л, М м, Н н, О о, П п, Р р, С с, Т т, У у, Ф ф, Х х, Ц ц, Ч ч, Ш ш, Щ щ, Ъ ъ, Ы ы, Ь ь, Э э, Ю ю, Я я

In [ ]:
tfidfvec=TfidfVectorizer()
dtm_tfidf_train=tfidfvec.fit_transform(train["description"])
dtm_tfidf_test = tfidfvec.transform(test["description"])
# get words
tfidfvec.get_feature_names()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(max_features=100, stop_words = stopWords, analyzer="word", token_pattern="[^0-9\s]{2,}")
tfidf = vec.fit_transform(train["description"])
potential_words=vec.get_feature_names()
len(potential_words)

In [ ]:
potential_words

In [ ]:
# translate keywords
from googletrans import Translator
def translate_ru_to_en(potential_words):
    translator = Translator()
    result=[translator.translate(origin, src='ru', dest='en').text for origin in potential_words]
    return result
    
    

def translate_title_and_description(file_path, prefix, title_only, desc_only):
    
    if title_only == 'yes':
        translate(file_path, prefix, "title")
    elif desc_only == 'yes':
        translate(file_path, prefix, "description")
    else:
        translate(file_path, prefix, "title")
        translate(file_path, prefix, "description")

In [ ]:
potential_words_ru=translate_ru_to_en(potential_words)

In [ ]:
for i in range(100):
    print("{}, {}".format(potential_words_ru[i], potential_words[i]))
    
# car, автомобиль
# big, большой
# house, дом, доме
# apartment, квартира
# flat, квартиру

# ideal, идеальном
#the new, новая
#new, новые
#new, новый
#excellent, отличное
#excellent, отличном
#highly, очень
#good, хорошее
#good, хорошем

#for sale, продается
#selling, продам
#selling, продаю
#for sale, продаётся
#bargain, торг
#bargain., торг.

# available, возможен
# guarantee, гарантия

## Check if keywords exist

In [17]:
def words_in_texts(words, texts):
    '''
    Args:
        words (list-like): words to find
        texts (Series): strings to search in
    
    Returns:
        NumPy array of 0s and 1s with shape (n, p) where n is the
        number of texts and p is the number of words.
    '''
    import re
    def getIndicator(email):
        
        lst=np.zeros(len(words))
        
        for i, word in enumerate(words):
            
            tmp=[]
            if len(re.findall(word, email))>0:
                
                lst[i]=1
            else:
                
                lst[i]=0
        return lst
     
    indicator=texts.apply(getIndicator)
    indicator_array = np.array(indicator.tolist())
    # YOUR CODE HERE
    
    return indicator_array

In [18]:
target_keyword=[# ideal, 
    "идеальном",
#the new, 
    "новая",
#new, 
    "новые",
#new, 
    "новый",
#excellent, 
    "отличное",
#excellent, 
    "отличном",
#highly, 
    "очень",
#good, 
    "хорошее",
#good, 
    "хорошем"]
potential_words_matrix=words_in_texts(target_keyword,train["description"])

In [19]:
df_potential_words=pd.DataFrame(potential_words_matrix, columns=target_keyword)

In [ ]:
def add_keywords_desc(data):
    
    return data

In [ ]:
keywords_matrix.shape

In [ ]:
df_potential_words

In [20]:
X_train=pd.concat([X_train, df_potential_words], axis=1)

In [ ]:
test_pd

In [21]:
X_train.head()

,item_id,title,description,char_len_title,char_len_desc,tfidf_title_0,tfidf_title_1,tfidf_title_2,tfidf_title_3,tfidf_title_4,...,has_desc,идеальном,новая,новые,новый,отличное,отличном,очень,хорошее,хорошем
0,b912c3c6a6ad,кокоби(кокон для сна),"кокон для сна малыша,пользовались меньше месяц...",21,58,0.00000,0.00000,0.00000,0.00000,0.00000,...,1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,2dac0150717d,стойка для одежды,"стойка для одежды, под вешалки. с бутика.",17,41,0.00000,0.00000,0.00000,0.00000,0.00000,...,1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,ba83aefab5dc,philips bluray,"в хорошем состоянии, домашний кинотеатр с blu ...",14,99,0.00000,0.00000,0.00000,0.00000,0.00000,...,1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
3,02996f1dd2ea,автокресло,продам кресло от0-25кг,10,22,0.00000,0.00000,0.00000,0.00000,0.00000,...,1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,7c90be56d2ab,"ваз 2110, 2003",все вопросы по телефону.,14,24,0.00000,0.00000,0.00000,0.00000,0.00000,...,1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
